In [2]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [12]:
# Load bike station data from previously exported .csv (See citybikes.ipynb)
station_data = pd.read_csv("../data/citybike_stations.csv")

# Format the latitude & longitude of each station for use in our API calls
station_coordinates = []
for index, row in station_data.iterrows():
    # Gather the lat and long for each station
    lat = row['latitude']
    lon = row['longitude']
    # Format the values into a single string
    formatted_coords = f"{lat},{lon}"
    # Add it to our coordinate list
    station_coordinates.append(formatted_coords)

# Headers & parameters needed for FourSquare
headers = {"Accept": "application/json"}
headers['Authorization'] = os.environ["FOURSQUARE_API_KEY"]
radius_to_search = 1000 # Distance of POI from bike station location (in meters)

# Send an API request for every pair of station coordinates to compile POI data
poi_data = []
for station_coord in station_coordinates:
    # Create the API call using the bike station location and provided search radius
    request_url = "https://api.foursquare.com/v3/places/search?ll=" + station_coord + "&radius=" + str(radius_to_search)
    response = requests.get(request_url, headers=headers)
    poi_data.append(response.json())

Load the API data into a DataFrame

In [4]:
# NOTE:seperate code block so I don't have to keep calling API
# There may be a better way to do the below, but I had a lot of issues getting this to work so I am leaving as-is :D

# Load the compiled data into Pandas dataframe 
poi_data_df = pd.DataFrame(poi_data)

# Use the sum function to flatten the results column (ie stop it from being a list of lists)
poi_data_flattened = sum(poi_data_df['results'], [])

# Load the organized data back into a dataframe
poi_data_df = pd.DataFrame(poi_data_flattened)

# Display results
poi_data_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London
1,4bc35be22a89ef3b08e7f488,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...",[],34,"{'main': {'latitude': 55.858078, 'longitude': ...",/v3/places/4bc35be22a89ef3b08e7f488,"{'address': '44 Candleriggs', 'admin_region': ...",The Dhabba,{},Europe/London
2,4b9cecbcf964a520bd8236e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],43,"{'main': {'latitude': 55.858429, 'longitude': ...",/v3/places/4b9cecbcf964a520bd8236e3,"{'address': '71-73 Albion St', 'admin_region':...",Merchant Square,{'children': [{'fsq_id': '595f841bf79faa623f93...,Europe/London
3,4ba0e8c3f964a520508637e3,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...",[],98,"{'main': {'latitude': 55.85886, 'longitude': -...",/v3/places/4ba0e8c3f964a520508637e3,"{'admin_region': 'Scotland', 'country': 'GB', ...",City Halls,{},Europe/London
4,554cdd3c498ee17d5a870117,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],144,"{'main': {'latitude': 55.859412, 'longitude': ...",/v3/places/554cdd3c498ee17d5a870117,"{'address': '127 Candleriggs', 'admin_region':...",Spitfire Espresso,{},Europe/London
...,...,...,...,...,...,...,...,...,...,...
1051,06b2e83c632e418806153b09,"[{'id': 17010, 'name': 'Used Car Dealership', ...",[],421,"{'main': {'latitude': 55.845601, 'longitude': ...",/v3/places/06b2e83c632e418806153b09,"{'address': '136 Boden St', 'admin_region': 'S...",London Cab Sales,{},Europe/London
1052,c6e7c00cef054a0a92e80063,"[{'id': 16037, 'name': 'Playground', 'icon': {...",[],543,"{'main': {'latitude': 55.842511, 'longitude': ...",/v3/places/c6e7c00cef054a0a92e80063,"{'address': '421 Baltic St', 'admin_region': '...",Baltic Street Adventure Playground,{},Europe/London
1053,4c1ae6aeb9f876b044847946,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],557,"{'main': {'latitude': 55.848546, 'longitude': ...",/v3/places/4c1ae6aeb9f876b044847946,"{'address': '851 London Rd', 'admin_region': '...",London Road Police Station,{},Europe/London
1054,5bc4936d898bdc002bfa5d46,"[{'id': 15003, 'name': 'Alternative Medicine C...",[],666,"{'drop_off': {'latitude': 55.840302, 'longitud...",/v3/places/5bc4936d898bdc002bfa5d46,"{'address': '45 Auckland Wynd', 'admin_region'...",Certified Professional Never Binge Again Maste...,{},Europe/London


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
# Copy the desired columns into the final dataframe
selected_poi_data_df = poi_data_df[['name','categories','location']].copy()

# Display results
selected_poi_data_df

,name,categories,location
0,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc..."
1,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ..."
2,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':..."
3,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ..."
4,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':..."
...,...,...,...
1051,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S..."
1052,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '..."
1053,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '..."
1054,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'..."


Extract the important values from the Category & Location dictionaries

In [6]:
# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['name'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address', '')
    
    return poi_address

# Use pandas .apply() dataframe function to extract the important values into new columns for much easier analysis
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
selected_poi_data_df['category_names'] = selected_poi_data_df.apply(get_category_names, axis=1)
selected_poi_data_df['address'] = selected_poi_data_df.apply(get_address, axis=1)

# Display the resulting DataFrame
selected_poi_data_df

,name,categories,location,category_names,address
0,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc...","(Café, Restaurant)",6 Wilson St
1,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ...","(Indian Restaurant,)",44 Candleriggs
2,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':...","(Bar, Restaurant)",71-73 Albion St
3,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ...","(Concert Hall, Theater)",
4,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':...","(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...,...,...
1051,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S...","(Used Car Dealership,)",136 Boden St
1052,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '...","(Playground,)",421 Baltic St
1053,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '...","(Police Station,)",851 London Rd
1054,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'...","(Alternative Medicine Clinic,)",45 Auckland Wynd


Drop the columns that are no longer needed & remove duplicates

In [7]:
# Drop columns
selected_poi_data_df = selected_poi_data_df.drop(['categories','location'], axis=1) # - Commented out as I've already run before and keep getting 'doesn't exist' errors

# Remove duplicates (Will happen in cases when two bike stations are within the search radius of eachother)
selected_poi_data_df = selected_poi_data_df.drop_duplicates()


# Final data frame to be exported to .CSV
selected_poi_data_df

,name,category_names,address
0,Wilson Street Pantry,"(Café, Restaurant)",6 Wilson St
1,The Dhabba,"(Indian Restaurant,)",44 Candleriggs
2,Merchant Square,"(Bar, Restaurant)",71-73 Albion St
3,City Halls,"(Concert Hall, Theater)",
4,Spitfire Espresso,"(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...
1042,Lidl,"(Grocery Store,)",19 Moss Rd
1043,Motus Commercials Glasgow,"(Car Dealership,)",131 Bogmoor Rd Govan
1044,Glasgow Centre For Reproductive Medicine,"(Healthcare Clinic,)",21 Fifty Pitches Way
1045,J K Electrical,"(Electrician,)","17 Fulbar Rd, Cardonald"


In [8]:
# Export to .csv
selected_poi_data_df.to_csv('../data/Foursquare_POIs.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:
# Headers & parameters needed for Yelp API acording to docs
# https://docs.developer.yelp.com/docs/fusion-intro
yelp_headers = {"accept": "application/json"}
yelp_headers['Authorization'] = "Bearer " + os.environ["YELP_API_KEY"]
radius_to_search = 1000

# Example API request url
#yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=55.8581672&longitude=-4.245483&radius=1000"

# Load bike station data from previously exported .csv (See citybikes.ipynb)
station_data = pd.read_csv('../data/citybike_stations.csv')

# Format the latitude & longitude of each station for use in our API calls
yelp_POIs = []
#api_limit_counter = 0 

# For each station in the loaded CSV
for index, row in station_data.iterrows():
    #if api_limit_counter >= 2: # Number of times I want to limit the API to run, used for testing
    #    break
    
    # Gather the lat and long for each station
    lat = row['latitude']
    lon = row['longitude']
    yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(lon) + "&radius=" + str(radius_to_search)
    response = requests.get(yelp_request_url, headers=yelp_headers)
    yelp_POIs.append(response.json()['businesses'])
    
    #api_limit_counter += 1

# Flatten the results, needed to properly format the list
yelp_POIs = sum(yelp_POIs,[])

Put your parsed results into a DataFrame

In [27]:
# Load the response into a dataframe
# NOTE: Use this cell to reset the DF instead of above to avoid unnessasary API calls
yelp_POIs_df = pd.DataFrame(yelp_POIs)
yelp_POIs_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,JNpi062fXUtd3lmGUSu-ww,café-gandolfi-glasgow-2,Café Gandolfi,https://s3-media3.fl.yelpcdn.com/bphoto/Zh1wQr...,False,https://www.yelp.com/biz/caf%C3%A9-gandolfi-gl...,67,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 55.8580469, 'longitude': -4.2442038}",[],££,"{'address1': '64 Albion Street', 'address2': '...",+441415526813,+44 141 552 6813,85.938424
1,FwZxNZrEYLdvBioEE76QVA,hutchesons-bar-and-brasserie-glasgow,Hutchesons Bar & Brasserie,https://s3-media4.fl.yelpcdn.com/bphoto/NYO7de...,False,https://www.yelp.com/biz/hutchesons-bar-and-br...,51,"[{'alias': 'scottish', 'title': 'Scottish'}, {...",4.0,"{'latitude': 55.859781774815, 'longitude': -4....",[],££,"{'address1': '158 Ingram Street', 'address2': ...",+441415524050,+44 141 552 4050,225.351596
2,ykFuZwPzHrXih4-Y4NAe4Q,brewdog-doghouse-glasgow,BrewDog DogHouse,https://s3-media4.fl.yelpcdn.com/bphoto/0iktlW...,False,https://www.yelp.com/biz/brewdog-doghouse-glas...,48,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 55.8593557, 'longitude': -4.2480296}",[],££,"{'address1': '99 Hutcheson Street', 'address2'...",+441415526363,+44 141 552 6363,206.258780
3,9I0QM3wBHiMzX2AvL3OQeg,the-wilson-street-pantry-glasgow,The Wilson Street Pantry,https://s3-media1.fl.yelpcdn.com/bphoto/G7Vz5a...,False,https://www.yelp.com/biz/the-wilson-street-pan...,88,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 55.8585085104567, 'longitude': -4...",[],£,"{'address1': '6 Wilson Street', 'address2': ''...",+441415520606,+44 141 552 0606,48.999836
4,UrnzTa6T3sLSVbbXLiYvQA,paesano-pizza-glasgow,Paesano Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/7KJazO...,False,https://www.yelp.com/biz/paesano-pizza-glasgow...,154,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 55.8596504377382, 'longitude': -4...",[],££,"{'address1': '94 Miller Street', 'address2': '...",+441412585565,+44 141 258 5565,359.689792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,K9O4skNFV5qiiLy5SYprAA,rab-haws-kitchens-glasgow,Rab Haws Kitchens,https://s3-media3.fl.yelpcdn.com/bphoto/IObET9...,False,https://www.yelp.com/biz/rab-haws-kitchens-gla...,2,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]",5.0,"{'latitude': 55.8499589, 'longitude': -4.1965876}",[],NaN,"{'address1': '111 Helenvale Street', 'address2...",+441415508811,+44 141 550 8811,834.579909
1998,nr09_r1igC6tvh-kpov30w,the-bank-glasgow-2,The Bank,,False,https://www.yelp.com/biz/the-bank-glasgow-2?ad...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",3.0,"{'latitude': 55.8510857, 'longitude': -4.1972446}",[],NaN,"{'address1': '1450 Gallowgate', 'address2': No...",+441415508999,+44 141 550 8999,868.350168
1999,zCU18QVRr0-Pz4M7qBQZ_Q,mcchans-glasgow,McChans,,False,https://www.yelp.com/biz/mcchans-glasgow?adjus...,1,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"{'latitude': 55.853449, 'longitude': -4.199841}",[],NaN,"{'address1': 'Unit 55 Forge Shopping Centre', ...",+441412371282,+44 141 237 1282,1012.326576
2000,s0ZVlYq5euYHpQEC8LL8YQ,bbs-coffee-and-muffins-glasgow-2,BB's Coffee and Muffins,https://s3-media2.fl.yelpcdn.com/bphoto/5aBsWM...,False,https://www.yelp.com/biz/bbs-coffee-and-muffin...,1,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 55.854661, 'longitude': -4.208116}",[],NaN,"{'address1': 'The Forge Shopping Centre', 'add...",+441415547878,+44 141 554 7878,1012.326576


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [31]:
# Copy over the columns to analyse
selected_yelp_POIs_df = yelp_POIs_df[['name','categories','location','rating']].copy()
selected_yelp_POIs_df

,name,categories,location,rating
0,Café Gandolfi,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'address1': '64 Albion Street', 'address2': '...",4.5
1,Hutchesons Bar & Brasserie,"[{'alias': 'scottish', 'title': 'Scottish'}, {...","{'address1': '158 Ingram Street', 'address2': ...",4.0
2,BrewDog DogHouse,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...","{'address1': '99 Hutcheson Street', 'address2'...",4.5
3,The Wilson Street Pantry,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'address1': '6 Wilson Street', 'address2': ''...",4.5
4,Paesano Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'address1': '94 Miller Street', 'address2': '...",4.5
...,...,...,...,...
1997,Rab Haws Kitchens,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]","{'address1': '111 Helenvale Street', 'address2...",5.0
1998,The Bank,"[{'alias': 'pubs', 'title': 'Pubs'}]","{'address1': '1450 Gallowgate', 'address2': No...",3.0
1999,McChans,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'address1': 'Unit 55 Forge Shopping Centre', ...",3.0
2000,BB's Coffee and Muffins,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'address1': 'The Forge Shopping Centre', 'add...",3.0


In [33]:
# Gather Addresss and Catagory names using the previously created functions used on the Foursquare data

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_yelp_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address1', '')
    
    return poi_address

# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_yelp_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['title'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple


selected_yelp_POIs_df['address'] = selected_yelp_POIs_df.apply(get_yelp_address, axis=1)
selected_yelp_POIs_df['category_names'] = selected_yelp_POIs_df.apply(get_yelp_category_names, axis=1)

selected_yelp_POIs_df

,name,categories,location,rating,address,category_names
0,Café Gandolfi,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'address1': '64 Albion Street', 'address2': '...",4.5,64 Albion Street,"(Seafood, British)"
1,Hutchesons Bar & Brasserie,"[{'alias': 'scottish', 'title': 'Scottish'}, {...","{'address1': '158 Ingram Street', 'address2': ...",4.0,158 Ingram Street,"(Scottish, Seafood, Steakhouses)"
2,BrewDog DogHouse,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...","{'address1': '99 Hutcheson Street', 'address2'...",4.5,99 Hutcheson Street,"(Pubs, Beer, Wine & Spirits, Barbeque)"
3,The Wilson Street Pantry,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'address1': '6 Wilson Street', 'address2': ''...",4.5,6 Wilson Street,"(Cafes, Coffee & Tea, Breakfast & Brunch)"
4,Paesano Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'address1': '94 Miller Street', 'address2': '...",4.5,94 Miller Street,"(Pizza, Italian)"
...,...,...,...,...,...,...
1997,Rab Haws Kitchens,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]","{'address1': '111 Helenvale Street', 'address2...",5.0,111 Helenvale Street,"(Fast Food,)"
1998,The Bank,"[{'alias': 'pubs', 'title': 'Pubs'}]","{'address1': '1450 Gallowgate', 'address2': No...",3.0,1450 Gallowgate,"(Pubs,)"
1999,McChans,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'address1': 'Unit 55 Forge Shopping Centre', ...",3.0,Unit 55 Forge Shopping Centre,"(Chinese,)"
2000,BB's Coffee and Muffins,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'address1': 'The Forge Shopping Centre', 'add...",3.0,The Forge Shopping Centre,"(Coffee & Tea,)"


In [34]:
# Drop columns that we have already parsed
selected_yelp_POIs_df = selected_yelp_POIs_df.drop(['categories','location'], axis=1)

# Remove duplicates
selected_yelp_POIs_df = selected_yelp_POIs_df.drop_duplicates()

# Final view before export to .csv
selected_yelp_POIs_df

,name,rating,address,category_names
0,Café Gandolfi,4.5,64 Albion Street,"(Seafood, British)"
1,Hutchesons Bar & Brasserie,4.0,158 Ingram Street,"(Scottish, Seafood, Steakhouses)"
2,BrewDog DogHouse,4.5,99 Hutcheson Street,"(Pubs, Beer, Wine & Spirits, Barbeque)"
3,The Wilson Street Pantry,4.5,6 Wilson Street,"(Cafes, Coffee & Tea, Breakfast & Brunch)"
4,Paesano Pizza,4.5,94 Miller Street,"(Pizza, Italian)"
...,...,...,...,...
1979,The Amethyst,4.0,1209 Govan Road,"(Pubs,)"
1980,Dowanhill Park,4.5,Havelock St/ Highburgh Rd,"(Playgrounds, Parks)"
1982,Roll With It,1.0,1145 Govan Road,"(Cafes, Delis, Soup)"
1983,Polka Dot Caffe,4.0,10 Holmfauldhead Place,"(Cafes,)"


In [35]:
# Export to .csv
selected_yelp_POIs_df.to_csv('../data/Yelp_POIs.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating